In [ ]:
%pip install requests

***We are scraping posts(links of posts) here and add them into a list in order to investigate further in the next steps.***

In [46]:
import requests
from bs4 import BeautifulSoup

urls = [
        'https://mitmachen.siegburg.de/angebotslandkarte',
        'https://mitmachen.jena.de/projekts'
    ]


def def_42(urls):
    urls_and_data = {}

    for url in urls:
        response = requests.get(url)
        if response.status_code != 200:
            print("Failed to retrieve the webpage. Status code:", response.status_code)

        soup = BeautifulSoup(response.content, 'html.parser')

        link_and_content = scraper(soup, url)
        urls_and_data[url] = link_and_content

    return urls_and_data


def scraper(soup, url):
    if 'siegburg' in url:
        return siegburg_data(soup)
    elif 'jena' in url:
        return jena_data(soup)
    
    return "No scraper defined for this URL"


def_42(urls)

{'https://mitmachen.siegburg.de/angebotslandkarte': {'https://mitmachen.siegburg.de/proposals/1090-senioiren-cafe-lichtblick-kaldauen': 'Senioiren Cafe Lichtblick Kaldauen18. September 20240 KommentareZugehöriges Projekt:\n            Angebotslandkarte60-7475+Geselliges BeisammenseinDas Cafe Lichtblick in Kaldauen lädt alle Senioren zum geselligen Beisammensein bei Kaffee und Kuchen ein. Es wird gesungen, gespielt, getanzt, gelacht und viel erzählt. Auch gemeinsame Ausflüge und Feiern stehen auf dem Programm.Haben Sie Interesse, dann kommen Sie gerne vorbei. Die Senioren und das Team freut sich auf Sie!Treffen : Alle 14 Tage Dienstags von 15.00 - 17.00 Uhr\xa0(immer in geraden Wochen)Ort : Pfarrheim der katholischen Kirche KaldauenAntoniusweg 1 ;\xa053721 SiegburgAnmeldungen bei Rita Quadt ; Tel.:\xa0015733725174:Kommentare(0)Um fortzufahren, müssen Sie sichanmeldenoderregistrieren.MCafé LichtblickBeigetreten am: 18.09.2024Konto verifiziertVorschläge1Unterstützer*innenUnterstützer*inne

In [45]:
def siegburg_data(soup):
    links = []

    a_tags = soup.find_all('a')

    for tag in a_tags:
        href = tag.get('href')
        
        if href and '/proposals/' in href and not 'new' in href:
            link = 'https://mitmachen.siegburg.de' + href
    
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'siegburg')

    return link_and_content


def jena_data(soup):
    links = []

    a_tags = soup.find_all('a',  class_='resource-item--title')

    for tag in a_tags:
        href = tag.get('href')
        
        if href:
            link = 'https://mitmachen.jena.de' + href
            
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'jena')

    return link_and_content

In [44]:
def content_scraper(soup, links, identifier):
    link_and_data = {}

    for link in links:
        response = requests.get(links[0])

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            match identifier:
                case 'jena':
                    title = soup.find_all('title')
                    content = soup.find_all('div', class_='flex-layout')

                    text_content = title[0].get_text(strip=True) + ': ' + content[0].get_text(strip=True)
                case 'siegburg':
                    divs = soup.find_all('div', class_='flex-layout')
                    content = divs[1]

                    text_content = content.get_text(strip=True)
                case _:
                    text_content = "Wrong identifier"

            link_and_data[link] = text_content

        else:
            message = "Failed to retrieve the webpage. Status code:", response.status_code
            link_and_data[link] = message
    
    return link_and_data